In [1]:
import json 
import requests
import pandas as pd 

In [ ]:
import requests
import pandas as pd
import os

API_KEY = os.getenv("TRIPADVISOR_API_KEY", "your-api-key-here")

countries = [
    "France", "Spain", "Italy", "Portugal", "Greece",
    "Morocco", "Thailand", "Mexico", "Japan", "USA",
    "Brazil", "Australia", "Indonesia", "Turkey", "Egypt"
]

headers = {
    "accept": "application/json",
    "Referer": "https://tripadvisor-content-api.readme.io/",
    "Origin": "https://tripadvisor-content-api.readme.io"
}

all_results = []

for country in countries:
    url = f"https://api.content.tripadvisor.com/api/v1/location/search?key={API_KEY}&searchQuery={country}&category=hotels&language=en"
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if "data" in data:
            for item in data["data"]:
                item["search_country"] = country
                all_results.append(item)
    
    print(f"{country}: {response.status_code} - {len(data.get('data', []))} results")

df = pd.DataFrame(all_results)
print(f"\nTotal: {len(all_results)} hotels")
df.head()

In [12]:
print(df.head())

  location_id                           name  \
0      152593                  Hotel Frances   
1      290407             Paris France Hotel   
2    13871137        Hotel Colline de France   
3     1700094         France Hotel Amsterdam   
4      487413  Hotel Campanile Val de France   

                                         address_obj search_country  
0  {'street1': 'Calle Maestranza 35', 'street2': ...         France  
1  {'street1': '72 Rue de Turbigo', 'street2': ''...         France  
2  {'street1': 'Rua Vigilante - Av.Central 400', ...         France  
3  {'street1': 'Oudezijds Kolk 11', 'street2': ''...         France  
4  {'street1': '10 Avenue de la Fosse des Pressoi...         France  


In [ ]:
import requests
import os

API_KEY = os.getenv("TRIPADVISOR_API_KEY", "your-api-key-here")

headers = {
    "accept": "application/json",
    "Referer": "https://tripadvisor-content-api.readme.io/",
    "Origin": "https://tripadvisor-content-api.readme.io"
}

all_photos = []

for location_id in df["location_id"]:
    url = f"https://api.content.tripadvisor.com/api/v1/location/{location_id}/photos?key={API_KEY}&language=en"
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if "data" in data:
            for photo in data["data"]:
                photo["location_id"] = location_id
                all_photos.append(photo)
    
    print(f"Location {location_id}: {response.status_code}")

df_photos = pd.DataFrame(all_photos)
print(f"\nTotal: {len(all_photos)} photos")
df_photos.head()

In [ ]:
# ====== FETCH REVIEWS ======
import requests
import os

API_KEY = os.getenv("TRIPADVISOR_API_KEY", "your-api-key-here")

headers = {
    "accept": "application/json",
    "Referer": "https://tripadvisor-content-api.readme.io/",
    "Origin": "https://tripadvisor-content-api.readme.io"
}

all_reviews = []

for location_id in df["location_id"]:
    url = f"https://api.content.tripadvisor.com/api/v1/location/{location_id}/reviews?key={API_KEY}&language=en&limit=10"
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if "data" in data:
            for review in data["data"]:
                review["location_id"] = location_id
                all_reviews.append(review)
    
    print(f"Location {location_id}: {response.status_code}")

df_reviews = pd.DataFrame(all_reviews)
print(f"\nTotal: {len(all_reviews)} reviews")
df_reviews.head()

In [ ]:
# ====== CONNEXION SUPABASE ======
from supabase import create_client, Client
import os

# Supabase local (valeurs de ton .env)
SUPABASE_URL = "http://127.0.0.1:54321"
SUPABASE_SERVICE_KEY = os.getenv("SUPABASE_SERVICE_KEY", "your-service-key-here")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_SERVICE_KEY)
print("Connected to Supabase")

In [16]:
# ====== INSERT LOCATIONS ======
import json

locations_to_insert = []

for _, row in df.iterrows():
    locations_to_insert.append({
        "location_id": str(row["location_id"]),
        "name": row["name"],
        "address_obj": row["address_obj"] if isinstance(row["address_obj"], dict) else json.loads(str(row["address_obj"]).replace("'", '"')),
        "search_country": row["search_country"],
        "category": "hotels"
    })

# Insert en batch
result = supabase.table("tripadvisor_locations").upsert(locations_to_insert).execute()
print(f"Inserted {len(locations_to_insert)} locations")

Inserted 150 locations


In [17]:
# ====== INSERT PHOTOS ======
photos_to_insert = []

for _, row in df_photos.iterrows():
    # Extraire les URLs des différentes tailles
    images = row.get("images", {})
    
    photos_to_insert.append({
        "location_id": str(row["location_id"]),
        "photo_id": str(row.get("id", "")),
        "url_original": images.get("original", {}).get("url", "") if isinstance(images, dict) else "",
        "url_large": images.get("large", {}).get("url", "") if isinstance(images, dict) else "",
        "url_medium": images.get("medium", {}).get("url", "") if isinstance(images, dict) else "",
        "url_small": images.get("small", {}).get("url", "") if isinstance(images, dict) else "",
        "caption": row.get("caption", ""),
        "uploaded_to_storage": False
    })

# Insert en batch
if photos_to_insert:
    result = supabase.table("tripadvisor_photos").upsert(photos_to_insert).execute()
    print(f"Inserted {len(photos_to_insert)} photos")
else:
    print("No photos to insert")

Inserted 726 photos


In [18]:
# ====== INSERT REVIEWS ======
reviews_to_insert = []

for _, row in df_reviews.iterrows():
    reviews_to_insert.append({
        "location_id": str(row["location_id"]),
        "review_id": str(row.get("id", "")),
        "rating": row.get("rating", None),
        "title": row.get("title", ""),
        "text": row.get("text", ""),
        "published_date": row.get("published_date", ""),
        "user_name": row.get("user", {}).get("username", "") if isinstance(row.get("user"), dict) else ""
    })

# Insert en batch
if reviews_to_insert:
    result = supabase.table("tripadvisor_reviews").upsert(reviews_to_insert).execute()
    print(f"Inserted {len(reviews_to_insert)} reviews")
else:
    print("No reviews to insert")

Inserted 577 reviews


In [20]:
# ====== UPLOAD IMAGES TO SUPABASE STORAGE ======
# (Optionnel - pour stocker les images localement)

import requests
from io import BytesIO

def upload_image_to_storage(image_url, location_id, photo_id):
    """Download image and upload to Supabase Storage"""
    try:
        # Download image
        response = requests.get(image_url, timeout=10)
        if response.status_code != 200:
            return None
        
        # Upload to storage
        file_path = f"tripadvisor/{location_id}/{photo_id}.jpg"
        
        supabase.storage.from_("images").upload(
            file_path,
            response.content,
            {"content-type": "image/jpeg"}
        )
        
        return file_path
    except Exception as e:
        print(f"Error uploading {photo_id}: {e}")
        return None

# Upload les images (limité à 10 pour test)
photos_result = supabase.table("tripadvisor_photos").select("*").limit(10).execute()

for photo in photos_result.data:
    if photo["url_medium"] and not photo["uploaded_to_storage"]:
        storage_path = upload_image_to_storage(
            photo["url_medium"],
            photo["location_id"],
            photo["photo_id"]
        )
        
        if storage_path:
            # Update record
            supabase.table("tripadvisor_photos").update({
                "uploaded_to_storage": True,
                "storage_path": storage_path
            }).eq("id", photo["id"]).execute()
            print(f"Uploaded: {storage_path}")

print("Done uploading images")

Uploaded: tripadvisor/152593/798809099.jpg
Uploaded: tripadvisor/152593/798809365.jpg
Uploaded: tripadvisor/152593/798809952.jpg
Uploaded: tripadvisor/152593/798809951.jpg
Uploaded: tripadvisor/152593/798809950.jpg
Uploaded: tripadvisor/290407/394332062.jpg
Uploaded: tripadvisor/290407/137551302.jpg
Uploaded: tripadvisor/290407/394337688.jpg
Uploaded: tripadvisor/290407/394336683.jpg
Uploaded: tripadvisor/290407/394337144.jpg
Done uploading images
